In [63]:
import numpy as np
import pandas as pd

In [64]:
# import data
df = pd.read_pickle('/Users/annabroido/Dropbox/Research/LRTAnalysis/LRTAnalysis/analysis/analysis.p')
df = df.query("Domain != 'Economic'")
df = df.query("alpha != ''")
# NOTE THAT 'IN' MEAN IN TO THE ALT DISTRIBUTION, WHCIH IS NOT WHAT MATHEMATICA DOES

In [65]:
def getcount(decision, direction, df=df):
    directiondir = {'in':'-1', 'out':'1', 'unk':'0', 'fail':'2'}
    query = '%s == %s' %(decision, directiondir[direction])
    num = len(np.asarray(df.query(query)[decision], dtype=int))
    return num

In [66]:
# plot some stuff
expin = getcount('dexp', 'in')
expout = getcount('dexp', 'out')
expunk = getcount('dexp', 'unk')
lnin = getcount('dln', 'in')
lnout = getcount('dln', 'out')
lnunk = getcount('dln', 'unk')
strexpin = getcount('dstrexp', 'in')
strexpout = getcount('dstrexp', 'out')
strexpunk = getcount('dstrexp', 'unk')
plwcin = getcount('dplwc', 'in')
plwcout = getcount('dplwc', 'out')
plwcunk = getcount('dplwc', 'unk')
# order = "LN", "Exp", "Str Exp", "PLwC"
incounts = [lnin, expin, strexpin, plwcin]
outcounts = [lnout, expout, strexpout, plwcout]
unkcounts = [lnunk, expunk, strexpunk, plwcunk]
print incounts
print outcounts
print unkcounts

[1396, 1066, 1291, 1702]
[479, 2336, 2020, 0]
[2511, 762, 1014, 2759]


In [67]:
unique_datasets = np.unique(df.fp_gml)
decisions = ['dln', 'dexp','dstrexp', 'dplwc']
#for i, dataset in enumerate(unique_datasets):
dataset = unique_datasets[0]
query = "fp_gml == '%s'" %dataset
rows = df.query(query)
incounts = np.zeros(4, dtype=int)
outcounts = np.zeros(4, dtype=int)
unkcounts = np.zeros(4, dtype=int)
for i, decision in enumerate(decisions):
    incounts[i] = getcount(decision, 'in', rows)
    outcounts[i] = getcount(decision, 'out', rows)
    unkcounts[i] = getcount(decision, 'unk', rows)
incounts

array([1, 2, 2, 1])

In [68]:
doms = np.unique(df.Domain)
# number of graphs and mean alpha value in each domain
counts = np.zeros_like(doms)
means = np.zeros_like(doms)
counts_plaus = np.zeros_like(doms)
means_plaus = np.zeros_like(doms)
counts_plaustail = np.zeros_like(doms)
means_plaustail = np.zeros_like(doms)
for i,dom in enumerate(doms):
    query = "Domain == '%s'" %dom
    subdf = df.query(query)
    counts[i] = len(subdf)
    means[i] = format(np.mean(subdf.alpha), '.2f')
    subdf_plaus = subdf.query("ppl>0.1")
    counts_plaus[i] = len(subdf_plaus)
    means_plaus[i] = format(np.mean(subdf_plaus.alpha), '.2f')
    subdf_plaustail = subdf_plaus.query("ntail>=50")
    counts_plaustail[i] = len(subdf_plaustail)
    means_plaustail[i] = format(np.mean(subdf_plaustail.alpha), '.2f')

In [69]:
print doms
print counts_plaustail
print means_plaustail

['Biological' 'Informational' 'Social' 'Technological' 'Transportation']
[125 13 239 1316 1]
['2.70' '2.69' '4.19' '2.19' '4.82']


In [70]:
def test_strong(rows):
    SA = False # strong alone
    S1 = False # strong 1
    S2 = False # strong 2
    n = len(rows)
    strong = 0
    strong_alone = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1 and row.ntail >= 50 and row.alpha < 3 and row.alpha > 2:
            strong_alone += 1
            if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1  and row.dplwc >-1:
                strong += 1
    if strong_alone >= 9.*n/10:
        SA = True
    if strong >= n/2.:
        S2 = True
    if SA == True and strong >= 95.*n/100:
        S1 = True
    return (S1, S2, SA)
            
def test_weak(rows):
    W = False 
    West = False 
    n = len(rows)
    weak = 0
    weakest = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1:
            weakest += 1
            if row.ntail>=50:
                weak += 1
    if weak >= n/2.:
        W = True
    if weakest >= n/2.:
        West = True
    return (W, West)
          

In [75]:
# test diff hypotheses
unique_datasets = np.unique(df.fp_gml)
hyps = pd.DataFrame(columns = ['Strong1', "Strong2", "Strong_alone", "Weak", "Weakest", "Domain", "Subdomain", "median_alpha", "n"], index=unique_datasets )
for i, dataset in enumerate(unique_datasets):
    query = "fp_gml == '%s'" %dataset
    rows = df.query(query)
    [S1, S2, SA] = test_strong(rows)
    hyps.loc[dataset]['Strong1'] = S1
    hyps.loc[dataset]['Strong2'] = S2
    hyps.loc[dataset]['Strong_alone'] = SA
    [weak,weakest] = test_weak(rows)
    hyps.loc[dataset]['Weak'] = weak
    hyps.loc[dataset]['Weakest'] = weakest
    hyps.loc[dataset]['Domain'] = rows.Domain[0]
    hyps.loc[dataset]['Subdomain'] = rows.Subdomain[0]
    hyps.loc[dataset]['median_alpha'] = np.median(rows.alpha)
    hyps.loc[dataset]['n'] = np.max(rows.n)

In [72]:
n = float(len(hyps))
Strong1 = np.sum(hyps.Strong1)
Strong2 = np.sum(hyps.Strong2)
Strong_alone = np.sum(hyps.Strong_alone)
Weak = np.sum(hyps.Weak)
Weakest = np.sum(hyps.Weakest)
print "Strong1 = %s" %(Strong1/n)
print "Strong2 = %s" %(Strong2/n)
print "Strong_alone = %s" %(Strong_alone/n)
print "Weak = %s" %(Weak/n)
print "Weakest = %s" %(Weakest/n)
print "Number of Datsets = %s" %int(n)

Strong1 = 0.0377562028047
Strong2 = 0.105717367853
Strong_alone = 0.0442286947141
Weak = 0.24163969795
Weakest = 0.333333333333
Number of Datsets = 927


In [22]:
hyps.to_csv('/Users/annabroido/Dropbox/Research/LRTAnalysis/LRTAnalysis/analysis/hyps.csv')

In [76]:
hyps

,Strong1,Strong2,Strong_alone,Weak,Weakest,Domain,Subdomain,median_alpha,n
/Volumes/Durodon/gmls/Biological/Connectome/n2/Cat_cerebral_hemisphere_cortex_and_thalamus_Biological_Connectome_n2.gml,False,False,False,False,True,Biological,Connectome,5.35,76
/Volumes/Durodon/gmls/Biological/Connectome/n2/Cat_cerebral_hemisphere_cortex_only_Biological_Connectome_n2.gml,False,False,False,False,True,Biological,Connectome,4.1,46
/Volumes/Durodon/gmls/Biological/Connectome/n2/Mouse_visual_cortex_2011_Mouse_Mouse_visual_cortex_1_Biological_Connectome_n2.gml,False,False,False,False,True,Biological,Connectome,4.4,29
/Volumes/Durodon/gmls/Biological/Connectome/n2/P_pacificus_nematode_2013_Worm_p_pacificus_neural_synaptic_1_Biological_Connectome_n2.gml,False,False,False,False,True,Biological,Connectome,1.91,54
/Volumes/Durodon/gmls/Biological/Connectome/n2/P_pacificus_nematode_2013_Worm_p_pacificus_neural_synaptic_2_Biological_Connectome_n2.gml,False,False,False,False,False,Biological,Connectome,1.71,50
/Volumes/Durodon/gmls/Biological/Connectome/n2/Rhesus_cortex_2013_Macaque_rhesus_interareal_cortical_network_2_Biological_Connectome_n2.gml,False,False,False,False,True,Biological,Connectome,1.99,88
/Volumes/Durodon/gmls/Biological/Connectome/n3/C_elegans_neurons_1986_C_elegans_Metabolic_Biological_Connectome_n3.gml,False,False,False,True,True,Biological,Connectome,3.29,297
/Volumes/Durodon/gmls/Biological/Connectome/n3/C_elegans_pharynx_2011_Worm_c_elegans_herm_pharynx_1_Biological_Connectome_n3.gml,False,False,False,True,True,Biological,Connectome,3.655,279
/Volumes/Durodon/gmls/Biological/Connectome/n3/Mouse_suprachiasmatic_nucleus_scn3_Biological_Connectome_n3.gml,False,False,False,False,True,Biological,Connectome,6.5,430
/Volumes/Durodon/gmls/Biological/Connectome/n3/Mouse_suprachiasmatic_nucleus_scn4_Biological_Connectome_n3.gml,False,False,False,False,True,Biological,Connectome,4.49,350


In [78]:
df.query("fp_gml == '/Volumes/Durodon/gmls/Transportation/Roads/n6/United_States_Road_Networks_PA_Transportation_Roads_n6.gml'")

,Domain,Subdomain,num_edges,Graph_order,Weighted,Directed,Bipartite,Multigraph,Multiplex,Component,...,n,alpha,xmin,ntail,Lpl,ppl,dexp,dln,dstrexp,dplwc
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplexroad_category_multigraphsimplifieddistribution.txt,Transportation,Roads,1078246,6,0,0,0,simplified,sub_road_category,entire,...,,6.3,3,456034,-291964,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplexroad_category_multigraphsimplified_largestcompdistribution.txt,Transportation,Roads,1072638,6,0,0,0,simplified,sub_road_category,largest,...,,6.3,3,455270,-291717,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplextravel_time_multigraphsimplifieddistribution.txt,Transportation,Roads,1078246,6,0,0,0,simplified,sub_travel_time,entire,...,,6.3,3,456034,-291964,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplextravel_time_multigraphsimplified_largestcompdistribution.txt,Transportation,Roads,1072638,6,0,0,0,simplified,sub_travel_time,largest,...,,6.3,3,455270,-291717,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplexspatial_distance_in_meters_multigraphsimplifieddistribution.txt,Transportation,Roads,1078246,6,0,0,0,simplified,sub_spatial_distance_in_meters,entire,...,,6.3,3,456034,-291964,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplexspatial_distance_in_meters_multigraphsimplified_largestcompdistribution.txt,Transportation,Roads,1072638,6,0,0,0,simplified,sub_spatial_distance_in_meters,largest,...,,6.3,3,455270,-291717,0,-1,-1,-1,-1


In [12]:
subhyps = hyps.query("Domain=='Social'")
n = float(len(subhyps))
Strong1 = np.sum(subhyps.Strong1)
Strong2 = np.sum(subhyps.Strong2)
Strong_alone = np.sum(subhyps.Strong_alone)
Weak = np.sum(subhyps.Weak)
Weakest = np.sum(subhyps.Weakest)
print "Strong1 = %s" %(Strong1/n)
print "Strong2 = %s" %(Strong2/n)
print "Strong_alone = %s" %(Strong_alone/n)
print "Weak = %s" %(Weak/n)
print "Weakest = %s" %(Weakest/n)
print "Number of Datsets = %s" %int(n)

Strong1 = 0.0
Strong2 = 0.0
Strong_alone = 0.0
Weak = 0.551724137931
Weakest = 0.696551724138
Number of Datsets = 145
